2. Automatic Document Clustering 

In [4]:
#cd OneDrive/Ross/CSC478/hw3/

C:\Users\johnr\OneDrive\Ross\CSC478\hw3


In [5]:
import numpy as np
import pandas as pd
import pylab as pl
import math
from sklearn.cluster import KMeans 
from sklearn.datasets import load_boston

a) Create your own distance function that, instead of using Euclidean distance, uses Cosine similarity. This is the distance function you will use to pass to the kMeans function.

In [6]:
def distCos(x, D):
    D_norm = np.linalg.norm(D)
    x_norm = np.linalg.norm(x)
    sim = np.dot(D,x)/(D_norm * x_norm)
    dist = 1 - sim
    return dist

b) Load the data set [Note: the data matrix provided has terms as rows and documents as columns. Since you will be clustering documents, you'll need to take the transpose of this matrix so that your main data matrix is a document x term matrix. In Numpy, you may use the ".T" operation to obtain the transpose.] Then, split the data set (the document x term matrix) and set aside 20% for later use (see below). Use the 80% segment for clustering in the next part. The 20% portion must be a random subset.

In [7]:
from sklearn.cross_validation import train_test_split

# create a list for the terms
terms = []
terms_list = open('terms.txt','r')

for t in terms_list:
    terms.append(t.strip())
terms_list.close()

# lod the data
data = np.genfromtxt('matrix.txt',delimiter=',')
# transpose the data so that it is in document x term format
data = data.T

#targets = pd.read_csv("classes.txt",skiprows=0, sep=' ')
targets = np.genfromtxt('classes.txt',skip_header=1,delimiter=' ')
# delete the index column from targets
targets = np.delete(targets, 0, 1) # 0th index, delete column (0) not row (1)

# split the data into 80-20 train-test
x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size=0.2, random_state=33)

c) As in the case of Assignment 2, transform the term-frequencies to tfxidf values. Be sure to maintain DF values for each of the terms in the dictionary. 

In [68]:
# Find doucment frequencies for each term
DF = np.array([(x_train!=0).sum(0)])
NDocs = len(x_train[0])
# Create a matrix with all entries = NDocs
NMatrix=np.ones(np.shape(x_train), dtype=float)*NDocs
# Convert each entry into IDF values
# Note that IDF is only a function of the term, so all rows will be identical.
DivM = np.divide(NMatrix, DF)
IDF = np.log2(DivM)
# Finally compute the TFxIDF values for each document-term entry
DT_tfidf = x_train * IDF

d) Perform Kmeans clustering on the training data. Write a function to display the top N terms in each cluster along with the cluster DF values for each term and the size of the cluster. Cluster DF value for a term t in a cluster C is the percentage of docs in cluster C in which term t appears. Sort the terms in decreasing order of the DF percentage.

In [71]:
import kMeans

centroids, clusterAssment = kMeans.kMeans(x_train, 5, distMeas=distCos)
#centroids, clusterAssment = kMeans.kMeans(DT_tfidf, 5, distMeas=distCos)

In [187]:
def display_terms(clusterAssment, centroids, x_train, terms, k):
    cluster_dict = {}
    for c in clusterAssment:
        if int(c[0]) in cluster_dict:
            cluster_dict[int(c[0])] += 1
        else:
            cluster_dict[int(c[0])] = 1
            
    cluster_count = 0     
    for cluster in cluster_dict:
        ind = np.argpartition(centroids[cluster_count], -k)[-k:]
        print 'Cluser ' + str(cluster_count) + ' results:\n'
        print 'Number of documents in cluster: ', cluster_dict[cluster_count]
        print '\tword\tDF\tPercentofDocs'
        
        word_count = 0
        for i in ind:
            term_found = np.array([(x_train[clusterAssment[:,0]==cluster_count][:,i]!=0).sum(0)])
            print str(word_count) + '\t' + terms[i] + '\t' + str(term_found[0]) + '\t' + str(term_found[0]*1.0 / cluster_dict[cluster_count]*1.0)
            word_count +=1
        print  
        cluster_count += 1
                


"\n        \n        print 'Number of documents in cluster: ', doc_count\n        cluster_count += 1\n"

In [189]:
display_terms(clusterAssment, centroids, x_train, terms, 10)

Cluser 0 results:

Number of documents in cluster:  766
	word	DF	PercentofDocs
0	run	141	0.18407310705
1	system	142	0.185378590078
2	work	180	0.23498694517
3	on	191	0.249347258486
4	program	135	0.176240208877
5	sale	210	0.274151436031
6	window	294	0.383812010444
7	do	152	0.198433420366
8	subject	766	1.0
9	file	158	0.206266318538

Cluser 1 results:

Number of documents in cluster:  6
	word	DF	PercentofDocs
0	tm	6	1.0
1	asq	6	1.0
2	mq	6	1.0
3	qq	6	1.0
4	giz	6	1.0
5	bxn	6	1.0
6	max	6	1.0
7	wm	6	1.0
8	ax	6	1.0
9	pl	6	1.0

Cluser 2 results:

Number of documents in cluster:  346
	word	DF	PercentofDocs
0	believ	144	0.416184971098
1	know	152	0.439306358382
2	subject	346	1.0
3	church	101	0.291907514451
4	db	1	0.0028901734104
5	peopl	164	0.473988439306
6	christian	173	0.5
7	jesu	104	0.300578034682
8	god	213	0.615606936416
9	on	208	0.601156069364

Cluser 3 results:

Number of documents in cluster:  855
	word	DF	PercentofDocs
0	encrypt	182	0.212865497076
1	go	287	0.33567251462
2	team	188	0.2198830

e) Using the cluster assignments from Kmeans clustering, compare your 5 clusters to the 5 pre-assigned classes by computing the Completeness and Homogeneity values.

In [212]:
from sklearn.metrics import completeness_score, homogeneity_score

y_train_int = y_train[:,0].astype(int)
c = clusterAssment[:,0].astype(int)

print completeness_score(y_train_int,c)
print homogeneity_score(y_train_int,c)

0.659525865273
0.454888372869


f) Using your cluster assignments as class labels, categorize each of the documents in the 20% set-aside data into each of the appropriate cluster. Your categorization should be based on Cosine similarity between each test document and each cluster centroids. Present your results in a separate file containing the obtained cluster label for each test document as well as Cosine similarities between each test document and each of the 5 clusters.

In [231]:
def knn_search(x, D):
    """ find nearest neighbour of data point test document x among cendtroids D """
    D_norm = np.array([np.linalg.norm(D[i]) for i in range(len(D))])
    x_norm = np.linalg.norm(x)
    sims = np.dot(D,x)/(D_norm * x_norm)
    dists = 1 - sims
    
    idx = np.argsort(dists) # sorting
    # return the index of nearest neighbor
    return idx[:1], dists

In [249]:
output_file = open('Ross Claytor CSC478 HW3 Part 2f Categorizations.txt','w')

for i in range(x_test.shape[0]):
    label, dists = knn_search(x_test[i], centroids)
    output_file.write('Test document ' + str(i+1) + '\tCluster Prediction: ' + str(label[0]) + ' Distances: ' + str(dists) + '\n')
output_file.close()